In [12]:
candidate = ['HBAR','GMT','XLM','STMX','1INCH','MTL','FLOW','SUI','BCH']
params = {5:{'market':5,'change_market':2,'spread':0,'ma':18,'vol_ma':12,'tp':0.15,'sl':0.03},
          20:{'market':20,'change_market':1,'spread':1,'ma':24,'vol_ma':12,'tp':0.3,'sl':0.03},
          50:{'market':50,'change_market':2,'spread':0,'ma':24,'vol_ma':16,'tp':0.15,'sl':0.03}}

In [ ]:
import os
import time
import pandas as pd

symbol_list = os.listdir('/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data')
market_table = pd.read_csv('/Users/johnsonhsiao/DC_capital/cmc_signal/data/market_table')
market_table.fillna(method='ffill',inplace=True)
market_table = market_table.set_index('time')

trade_record = []
for symbol in symbol_list:  
    param_map = {}
    
    if symbol == '.DS_Store':
        pass
    else:        
        for index in market_table.index:
            if  5 <= market_table[symbol].loc[index] < 20:
                param = params[5]
            elif 20 <= market_table[symbol].loc[index] < 50:
                param = params[20]
            elif 50 < market_table[symbol].loc[index]:
                param = params[50]
            param_map[index] = param
            
        print(symbol)
        test = bt_data_handler_v2(symbol)
        record = backtest(symbol,test,param_map)
        trade_record.append(record)
        if len(record) != 0:
            print(f'pnl = {record[-1][-2]}')
    
flattened_data = [trade for sublist in trade_record for trade in sublist]
trade_record_df = pd.DataFrame(pd.DataFrame(flattened_data, columns=['symbol','time','price','direction','pnl','totol return','trade times']))
trade_record_df.to_csv('/Users/johnsonhsiao/DC_capital/cmc_signal/data/record/params')


CELO
'2023-04-25 00:15:00'
'2023-07-20 15:00:00'
pnl = 0.3495023397605049
ADA
'2023-04-25 00:15:00'


In [21]:
def backtest(symbol,test,param_map):    
    df = test
    position = 0
    pnl = 0
    trade = 0
    trades = []
    
    for index in df.index:
        try:
            tp = param_map[f'{index}']['tp']
            sl = param_map[f'{index}']['sl']
            market = param_map[f'{index}']['market']
            change_market = param_map[f'{index}']['change_market']
            spread = param_map[f'{index}']['spread']
            ma = param_map[f'{index}']['ma']
            vol_ma = param_map[f'{index}']['vol_ma']

            df['ma'] = df['up_close'].shift(1).rolling(window = ma).mean()
            df['uband'] = df['ma'] + 1*df['up_close'].shift(1).rolling(window = ma).std()
            df['lband'] = df['ma'] - 1*df['up_close'].shift(1).rolling(window = ma).std() 

            df['vol_ma'] = df['up_1_hour'].rolling(window = vol_ma).mean()
            df['vol_uband'] = df['vol_ma'] + 2*df['up_1_hour'].rolling(window = vol_ma).std()
            df['vol_lband'] = df['vol_ma'] - 2*df['up_1_hour'].rolling(window = vol_ma).std() 

            ub_market = df['upbit/market'].loc[index] > market # upbit佔全市場
            spread_n = df['spread'].loc[index] > spread #test['spread'] > 1 #價差 > 2%
            spread_20 = df['spread'].loc[index] < 20 #價差不能太大，有些幣種差太多
            ub_market_pct = df['%upbit/market'].loc[index] > change_market #upbit佔全市場的變化量
            low_val = df['up_close'].loc[index] < df['uband'].loc[index] #波動不能太大
            vol_test = df['up_1_hour'].loc[index] > df['vol_uband'].loc[index]

            entry = ub_market & spread_n & spread_20 & ub_market_pct & low_val & vol_test

            if position == 1:
                if ((df['up_close'].loc[index] - entry_price) / entry_price) > tp:
                    pnl += (df['up_close'].loc[index] - entry_price) / entry_price
                    position_pnl = (df['up_close'].loc[index] - entry_price) / entry_price
                    trade += 1
                    position = 0
                    trades.append([symbol,index,df['up_close'].loc[index],'take profit',position_pnl,pnl,trade])

                if ((df['up_close'].loc[index] - entry_price) / entry_price) < -sl:
                    pnl += (df['up_close'].loc[index] - entry_price) / entry_price
                    position_pnl = (df['up_close'].loc[index] - entry_price) / entry_price
                    trade += 1
                    position = 0
                    trades.append([symbol,index,df['up_close'].loc[index],'stop loss',position_pnl,pnl,trade])
                   
            if entry == True and position == 0:
                trade += 1
                position = 1
                entry_price = df['up_close'].loc[index]
                position_pnl = 0
                trades.append([symbol,index,entry_price,'long position',position_pnl,pnl,trade])
            
        except Exception as e:
            print(e)
        
    if position == 1:
        position_pnl = (df['up_close'].loc[index] - entry_price) / entry_price
        trades.append([symbol,index,entry_price,'unclosed position',position_pnl,pnl,trade])
            
    return trades

### 基本數據

In [7]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
def bt_data_handler_v2(symbol):
    
    cmc = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/cmc_15m_data/{symbol}').drop(columns = 'Unnamed: 0')
    bn_raw = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/bn_15m_data/{symbol}USDT').drop(columns = 'Unnamed: 0')
    upbit_raw = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data/{symbol}').drop(columns = 'Unnamed: 0')
    cmc['time'] = pd.to_datetime(cmc['time'])
    cmc['time'] = cmc['time'].dt.tz_localize(None)
    
    if symbol == 'IOTA':
        cmc = cmc.rename(columns={'MIOTA':'market'})
    else:
        cmc = cmc.rename(columns={f'{symbol}':'market'})

    bn_raw['time'] = pd.to_datetime(bn_raw['timestamp'])
    bn_raw = bn_raw.drop(columns=['timestamp','ignore','close_time'])
    bn = pd.DataFrame()
    bn[['time','bn_open','bn_high','bn_low','bn_close','bn_volume','bn_quote']] = bn_raw[['time','open','high','low','close','volume','quote_asset_volume']]

    upbit_raw['time'] = pd.to_datetime(upbit_raw['candle_date_time_utc'])
    upbit = pd.DataFrame()
    upbit[['time','up_open','up_high','up_low','up_close','up_volume','up_quote']] = upbit_raw[['time','opening_price','high_price','low_price','trade_price','candle_acc_trade_volume','candle_acc_trade_price']]
    
    merged_df = pd.merge(bn, upbit, on='time', how='inner')
    merged_df = pd.merge(merged_df, cmc, on='time', how='inner')
    sorted_df = merged_df.sort_values(by='time', ascending=True)
    test = sorted_df.dropna().reset_index(drop=True)

    test['up_24_hour'] = test['up_volume'].shift(1).rolling(window=96).sum()
    test['up_12_hour'] = test['up_volume'].shift(1).rolling(window=48).sum()
    test['up_6_hour'] = test['up_volume'].shift(1).rolling(window=24).sum()
    test['up_4_hour'] = test['up_volume'].shift(1).rolling(window=16).sum()
    test['up_1_hour'] = test['up_volume'].shift(1).rolling(window=4).sum()
    
    test['bn_24_hour'] = test['bn_volume'].shift(1).rolling(window=96).sum()
    test['bn_12_hour'] = test['bn_volume'].shift(1).rolling(window=48).sum()
    test['bn_6_hour'] = test['bn_volume'].shift(1).rolling(window=24).sum()
    test['bn_4_hour'] = test['bn_volume'].shift(1).rolling(window=16).sum()
    test['bn_1_hour'] = test['bn_volume'].shift(1).rolling(window=4).sum()

    test = test.dropna()
    test = test.reset_index()
    
    krw_usdt = pd.read_csv('/Users/johnsonhsiao/DC_capital/cmc_signal/USDT-KRW.csv')
    krw_usdt['Date'] = pd.to_datetime(krw_usdt['Date'])
    krw_usdt['krw_usdt'] = (krw_usdt['Open']+krw_usdt['High']+krw_usdt['Low']+krw_usdt['Close']) / 4
    krw_usdt = krw_usdt.drop(columns=['Open','High','Low','Close','Adj Close','Volume'])
    test['Date'] = test['time']
    test = test.merge(krw_usdt, on='Date', how='left')
    test['krw_usdt'].fillna(method='ffill', inplace=True)
    test = test.drop(columns=['Date','index'])
    test = test.set_index('time')
    
    test['up_close'] = test['up_close'] / test['krw_usdt']
    test['upbit/market'] = (test['up_24_hour'].shift(1) * test['up_close'].shift(1) / test['market'].shift(1)) * 100
    test['%upbit/market'] = test['upbit/market'].pct_change()*100
    
    test['spread'] = (test['up_close'] / test['bn_close'] -1) * 100
    test['up1h%'] = test['up_1_hour'].pct_change()*100
    test['bn1h%'] = test['bn_1_hour'].pct_change()*100
    
    return test

# 製作market截面表

In [3]:
import os
import pandas as pd
symbol_list = os.listdir('/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data')
symbol_list.remove('.DS_Store')
data = pd.DataFrame()
a = 0
for symbol in symbol_list:
    
    cmc = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/cmc_15m_data/{symbol}').drop(columns = 'Unnamed: 0')
    upbit_raw = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data/{symbol}').drop(columns = 'Unnamed: 0')
    cmc['time'] = pd.to_datetime(cmc['time'])
    cmc['time'] = cmc['time'].dt.tz_localize(None)

    upbit_raw['time'] = pd.to_datetime(upbit_raw['candle_date_time_utc'])
    upbit = pd.DataFrame()
    upbit[['time','up_open','up_high','up_low','up_close','up_volume','up_quote']] = upbit_raw[['time','opening_price','high_price','low_price','trade_price','candle_acc_trade_volume','candle_acc_trade_price']]
        
    merged_df = pd.merge(upbit, cmc, on='time', how='inner')
    sorted_df = merged_df.sort_values(by='time', ascending=True)
    test = sorted_df.dropna()
    
    krw_usdt = pd.read_csv('/Users/johnsonhsiao/DC_capital/cmc_signal/USDT-KRW.csv')
    krw_usdt['Date'] = pd.to_datetime(krw_usdt['Date'])
    krw_usdt['krw_usdt'] = (krw_usdt['Open']+krw_usdt['High']+krw_usdt['Low']+krw_usdt['Close']) / 4
    krw_usdt = krw_usdt.drop(columns=['Open','High','Low','Close','Adj Close','Volume'])
    test['Date'] = test['time']
    test = test.merge(krw_usdt, on='Date', how='left')
    test['krw_usdt'].fillna(method='ffill', inplace=True)
    test = test.drop(columns=['Date'])
    test = test.set_index('time')
    
    if symbol == 'IOTA':
        test['IOTA'] = test['MIOTA']
    
    test['up_close'] = test['up_close'] / test['krw_usdt']
    test['up_24_hour'] = test['up_volume'].shift(1).rolling(window=96).sum()
    test['upbit/market'] = (test['up_24_hour'].shift(1) * test['up_close'].shift(1) / test[symbol].shift(1)) * 100
    test = test.dropna()
    
    if symbol == 'IOTA':
        data['IOTA'] = test['upbit/market']
    else:
        data[symbol] = test['upbit/market']
    
data.to_csv('/Users/johnsonhsiao/DC_capital/cmc_signal/data/market_table')

# Origin Version

In [1086]:
import os
symbol_list = os.listdir('/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data')
sample = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data/BTC').drop(columns = 'Unnamed: 0')

top_20 = pd.DataFrame()

for symbol in symbol_list:
    if symbol == '.DS_Store':
        pass
    else:
        upbit = pd.read_csv(f'/Users/johnsonhsiao/DC_capital/cmc_signal/data/upbit_15m_data/{symbol}').drop(columns = 'Unnamed: 0')
        upbit['time'] = pd.to_datetime(sample['candle_date_time_utc'])
        upbit = upbit.set_index('time')
        top_20[symbol] = upbit['candle_acc_trade_price']
        
top_20 = top_20.T
top_20 = top_20.rank(axis=0)
top_20 = top_20.fillna(0)

In [1113]:
import matplotlib.pyplot as plt

def plooter(test,tp,sl):
    
    record = []
    
    df = test[-4000:]
    fig = plt.figure(figsize=(12, 6))

    ax1 = fig.add_subplot()
    ax1.plot(df.index, df['up_close'], color='blue', label='upbit_price')
    ax1.set_ylabel('up_price', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')

    ax2 = ax1.twinx()
    ax2.plot(df.index, df['spread'],alpha = 0.3 ,color='black', label='spread')
    ax2.set_ylabel('spread', color='black')
    ax2.tick_params(axis='y', labelcolor='black')

    ax3 = ax1.twinx()
    ax3.plot(df.index, df['up_1_hour'], alpha=0.5, color='orange', label='volume')
    ax3.set_ylabel('volume', color='orange')
    ax3.tick_params(axis='y', labelcolor='orange')
    
    ax1.legend(loc='best')
    ax2.legend(loc='best')
    ax3.legend(loc='best')       
    
    position = 0
    pnl = 0
    trade = 0
        
    for index, row in df.iterrows():
        if position == 1:
            if ((row['up_close'] - entry_price) / entry_price) > tp:
                pnl += (row['up_close'] - entry_price) / entry_price
                trade += 1
                position -= 1
                ax1.plot(index, row['up_close'], 'rv', markersize=10)
            if ((row['up_close'] - entry_price) / entry_price) < -sl:
                pnl += (row['up_close'] - entry_price) / entry_price
                trade += 1
                position -= 1
                ax1.plot(index, row['up_close'], 'ro', markersize=10)
                
        if row['entry'] == True and position == 0:
            trade += 1
            position += 1
            ax1.plot(index, row['up_close'], 'g^', markersize=10)
            entry_price = row['up_close']
            
    print(f'pnl:{pnl*100}%')
    print(f'total trade:{trade}')
    print(f'tp:{tp*100}%')
    print(f'sl:{sl*100}%')
    plt.xticks(rotation=45)

    plt.show()
    return pnl


In [1131]:
for symbol in symbol_list:
    test = bt_data_handler_v2(symbol)
    

In [ ]:
origin_params = [{'symbol':'HBAR','market':5,'change_market':5,'spread':0,'ma':18,'vol_ma':8,'tp':0.05,'sl':0.03},
          {'symbol':'XLM','market':5,'change_market':2,'spread':0,'ma':18,'vol_ma':12,'tp':0.35,'sl':0.035},
          {'symbol':'GMT','market':25,'change_market':0.5,'spread':1,'ma':24,'vol_ma':12,'tp':0.15,'sl':0.03},
          {'symbol':'1INCH','market':15,'change_market':6,'spread':0,'ma':24,'vol_ma':12,'tp':0.6,'sl':0.06},
          {'symbol':'BCH','market':20,'change_market':1,'spread':1,'ma':18,'vol_ma':8,'tp':0.7,'sl':0.07},
          {'symbol':'MTL','market':50,'change_market':0.5,'spread':2,'ma':24,'vol_ma':16,'tp':0.2,'sl':0.05},
          {'symbol':'FLOW','market':50,'change_market':3,'spread':0,'ma':26,'vol_ma':16,'tp':0.4,'sl':0.03},
          {'symbol':'STMX','market':50,'change_market':2,'spread':0,'ma':48,'vol_ma':16,'tp':0.7,'sl':0.07}]